In [1]:
# Install a pip package in the current Jupyter kernel - REMOVE IN PROD
import sys
!{sys.executable} -m pip install gtfparse

  Created wheel for gtfparse: filename=gtfparse-1.2.0-py3-none-any.whl size=11881 sha256=b5c9914eb867c3dee28f96fa30d73dafb706f1029dedcd8a40c8908e852673d8
  Stored in directory: /home/jovyan/.cache/pip/wheels/2d/81/83/85c4108d53b4b2ad3cee5d6f9b7ec612554e132202b8020359
Successfully built gtfparse


In [2]:
# Import libs
import os.path
from os import path
import gzip
import pandas as pd
import urllib.request
from gtfparse import read_gtf

In [3]:
# Setup
data_path = '../data'
gtf_path_gz = data_path + '/Mus_musculus.GRCm38.99.gtf.gz'
gtf_path = data_path + '/Mus_musculus.GRCm38.99.gtf'

In [4]:
# Download initial datasets and decompress
if path.exists(gtf_path_gz) is False: 
    urllib.request.urlretrieve('ftp://ftp.ensembl.org/pub/release-99/gtf/mus_musculus/Mus_musculus.GRCm38.99.gtf.gz', gtf_path_gz)
    input = gzip.GzipFile(gtf_path_gz, 'rb')
    s = input.read()
    input.close()
    output = open(gtf_path, 'wb')
    output.write(s)
    output.close()

In [8]:
df_gtf = read_gtf(gtf_path)

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'ccds_id', 'protein_id', 'protein_version']


In [9]:
df_gtf

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,transcript_source,transcript_biotype,tag,transcript_support_level,exon_number,exon_id,exon_version,ccds_id,protein_id,protein_version
0,1,havana,gene,3073253,3074322,NaN,+,0,ENSMUSG00000102693,1,...,,,,,,,,,,
1,1,havana,transcript,3073253,3074322,NaN,+,0,ENSMUSG00000102693,1,...,havana,TEC,basic,NA,,,,,,
2,1,havana,exon,3073253,3074322,NaN,+,0,ENSMUSG00000102693,1,...,havana,TEC,basic,NA,1,ENSMUSE00001343744,1,,,
3,1,ensembl,gene,3102016,3102125,NaN,+,0,ENSMUSG00000064842,1,...,,,,,,,,,,
4,1,ensembl,transcript,3102016,3102125,NaN,+,0,ENSMUSG00000064842,1,...,ensembl,snRNA,basic,NA,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868325,JH584295.1,ensembl,CDS,708,752,NaN,-,2,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,5,,,,ENSMUSP00000137004,1
1868326,JH584295.1,ensembl,exon,565,633,NaN,-,0,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,6,ENSMUSE00000997159,1,,,
1868327,JH584295.1,ensembl,CDS,565,633,NaN,-,2,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,6,,,,ENSMUSP00000137004,1
1868328,JH584295.1,ensembl,exon,66,109,NaN,-,0,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,7,ENSMUSE00001007635,1,,,


In [13]:
df_gtf_filt = df_gtf[(df_gtf.feature=='exon') | (df_gtf.feature=='three_prime_utr') | (df_gtf.feature=='five_prime_utr')]

In [15]:
df_gtf_filt

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,transcript_source,transcript_biotype,tag,transcript_support_level,exon_number,exon_id,exon_version,ccds_id,protein_id,protein_version
2,1,havana,exon,3073253,3074322,NaN,+,0,ENSMUSG00000102693,1,...,havana,TEC,basic,NA,1,ENSMUSE00001343744,1,,,
5,1,ensembl,exon,3102016,3102125,NaN,+,0,ENSMUSG00000064842,1,...,ensembl,snRNA,basic,NA,1,ENSMUSE00000522066,1,,,
8,1,havana,exon,3213609,3216344,NaN,-,0,ENSMUSG00000051951,5,...,havana,processed_transcript,,1,1,ENSMUSE00000858910,1,,,
9,1,havana,exon,3205901,3207317,NaN,-,0,ENSMUSG00000051951,5,...,havana,processed_transcript,,1,2,ENSMUSE00000866652,1,,,
11,1,havana,exon,3213439,3215632,NaN,-,0,ENSMUSG00000051951,5,...,havana,processed_transcript,,1,1,ENSMUSE00000863980,1,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868320,JH584295.1,ensembl,exon,986,1009,NaN,-,0,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,3,ENSMUSE00001085113,1,,,
1868322,JH584295.1,ensembl,exon,972,981,NaN,-,0,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,4,ENSMUSE00001070510,1,,,
1868324,JH584295.1,ensembl,exon,708,752,NaN,-,0,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,5,ENSMUSE00001015770,1,,,
1868326,JH584295.1,ensembl,exon,565,633,NaN,-,0,ENSMUSG00000095742,1,...,ensembl,protein_coding,basic,5,6,ENSMUSE00000997159,1,,,
